# Step 2. Angles extraction

In [10]:
import os
import math
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp

In [11]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(
    static_image_mode=True,
    min_detection_confidence=0.3,
    model_complexity=2
)

I0000 00:00:1764990377.545593 3036324 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro


In [12]:
def detectPose(image, pose):
    """
    Run MediaPipe pose on an image and return:
        output_image: image with landmarks drawn
        landmarks: list of (x, y, z) in pixel space.
    """
    output_image = image.copy()
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(imageRGB)

    height, width, _ = image.shape
    landmarks = []

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image=output_image,
            landmark_list=results.pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS
        )
        for lm in results.pose_landmarks.landmark:
            landmarks.append((
                int(lm.x * width),
                int(lm.y * height),
                lm.z * width
            ))

    return output_image, landmarks


def calculateAngle(landmark1, landmark2, landmark3):
    """
    Angle between three keypoints (in 2D) at landmark2.
    """
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    angle = math.degrees(
        math.atan2(y3 - y2, x3 - x2) -
        math.atan2(y1 - y2, x1 - x2)
    )

    if angle < 0:
        angle += 360
    return angle

def angles_finder(landmarks):
    """
    Compute a set of angles from a list of landmarks.
    Returns a list of 14 angles in a fixed order.
    """
    # for brevity, alias the enum
    P = mp_pose.PoseLandmark

    left_elbow_angle = calculateAngle(
        landmarks[P.LEFT_SHOULDER.value],
        landmarks[P.LEFT_ELBOW.value],
        landmarks[P.LEFT_WRIST.value]
    )
    right_elbow_angle = calculateAngle(
        landmarks[P.RIGHT_SHOULDER.value],
        landmarks[P.RIGHT_ELBOW.value],
        landmarks[P.RIGHT_WRIST.value]
    )

    left_shoulder_angle = calculateAngle(
        landmarks[P.LEFT_ELBOW.value],
        landmarks[P.LEFT_SHOULDER.value],
        landmarks[P.LEFT_HIP.value]
    )
    right_shoulder_angle = calculateAngle(
        landmarks[P.RIGHT_HIP.value],
        landmarks[P.RIGHT_SHOULDER.value],
        landmarks[P.RIGHT_ELBOW.value]
    )

    left_knee_angle = calculateAngle(
        landmarks[P.LEFT_HIP.value],
        landmarks[P.LEFT_KNEE.value],
        landmarks[P.LEFT_ANKLE.value]
    )
    right_knee_angle = calculateAngle(
        landmarks[P.RIGHT_HIP.value],
        landmarks[P.RIGHT_KNEE.value],
        landmarks[P.RIGHT_ANKLE.value]
    )

    hand_angle = calculateAngle(
        landmarks[P.LEFT_ELBOW.value],
        landmarks[P.RIGHT_SHOULDER.value],
        landmarks[P.RIGHT_ELBOW.value]
    )

    left_hip_angle = calculateAngle(
        landmarks[P.LEFT_SHOULDER.value],
        landmarks[P.LEFT_HIP.value],
        landmarks[P.LEFT_KNEE.value]
    )
    right_hip_angle = calculateAngle(
        landmarks[P.RIGHT_SHOULDER.value],
        landmarks[P.RIGHT_HIP.value],
        landmarks[P.RIGHT_KNEE.value]
    )

    neck_angle_uk = calculateAngle(
        landmarks[P.NOSE.value],
        landmarks[P.LEFT_SHOULDER.value],
        landmarks[P.RIGHT_SHOULDER.value]
    )

    left_wrist_angle_bk = calculateAngle(
        landmarks[P.LEFT_WRIST.value],
        landmarks[P.LEFT_HIP.value],
        landmarks[P.LEFT_ANKLE.value]
    )
    right_wrist_angle_bk = calculateAngle(
        landmarks[P.RIGHT_WRIST.value],
        landmarks[P.RIGHT_HIP.value],
        landmarks[P.RIGHT_ANKLE.value]
    )

    return [
        left_elbow_angle, right_elbow_angle,
        left_shoulder_angle, right_shoulder_angle,
        left_knee_angle, right_knee_angle,
        hand_angle,
        left_hip_angle, right_hip_angle,
        neck_angle_uk,
        left_wrist_angle_bk, right_wrist_angle_bk
    ]

In [13]:
POSES = ["Downdog", "Goddess", "Plank", "Tree", "Warrior2"]

data_path = "../../data_clean"
angles_path = "../angles_clean"

for pose_name in POSES:
    path = f"{data_path}/{pose_name}/Images"

    columns = [
        "pose_label",
        "image_path",
        "left_elbow_angle", "right_elbow_angle",
        "left_shoulder_angle", "right_shoulder_angle",
        "left_knee_angle", "right_knee_angle",
        "hand_angle",
        "left_hip_angle", "right_hip_angle",
        "neck_angle_uk",
        "left_wrist_angle_bk", "right_wrist_angle_bk"
    ]

    df = pd.DataFrame(columns=columns)

    for filename in os.listdir(path):
        if not (filename.lower().endswith(".jpg")
                or filename.lower().endswith(".jpeg")
                or filename.lower().endswith(".png")):
            continue

        img_path = os.path.join(path, filename)
        image = cv2.imread(img_path)
        if image is None:
            print(f"[WARN] Could not read {img_path}")
            continue

        output_image, landmarks = detectPose(image, pose)
        if landmarks:
            angles = angles_finder(landmarks)
            row = {
                "pose_label": pose_name,
                "image_path": img_path,
                "left_elbow_angle": angles[0],
                "right_elbow_angle": angles[1],
                "left_shoulder_angle": angles[2],
                "right_shoulder_angle": angles[3],
                "left_knee_angle": angles[4],
                "right_knee_angle": angles[5],
                "hand_angle": angles[6],
                "left_hip_angle": angles[7],
                "right_hip_angle": angles[8],
                "neck_angle_uk": angles[9],
                "left_wrist_angle_bk": angles[10],
                "right_wrist_angle_bk": angles[11],
            }
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

    print(df.head())

    os.makedirs(f"{angles_path}", exist_ok=True)
    out_csv = f"{angles_path}/{pose_name}.csv"
    df.to_csv(out_csv, index=False)
    print("Saved angles CSV to:", out_csv)

W0000 00:00:1764990377.637277 3056336 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764990377.680560 3056336 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/qz/Desktop/Computer Vision/Deep-Learning-Fitness-Poses-Correction-/yoga_env/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/var/folders/_p/cwjtcdzj0w37y_rj10wlyyq00000gn/T/ipykernel_76545/3577633952.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA col

  pose_label                                    image_path  left_elbow_angle  \
0    Downdog  ../../data_clean/Downdog/Images/00000372.jpg        195.382277   
1    Downdog  ../../data_clean/Downdog/Images/00000414.jpg        161.394529   
2    Downdog  ../../data_clean/Downdog/Images/00000158.jpg        188.316559   
3    Downdog  ../../data_clean/Downdog/Images/00000164.jpg        163.143669   
4    Downdog  ../../data_clean/Downdog/Images/00000170.jpg        202.094230   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         189.994100           181.919382            180.159746   
1         158.980446           163.869613            198.508847   
2         189.387117           188.434129            168.625045   
3         151.368673           174.269075            190.190288   
4         196.515703           181.847610            181.956953   

   left_knee_angle  right_knee_angle  hand_angle  left_hip_angle  \
0       153.536511        156.576545    1.285724

/Users/qz/Desktop/Computer Vision/Deep-Learning-Fitness-Poses-Correction-/yoga_env/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/var/folders/_p/cwjtcdzj0w37y_rj10wlyyq00000gn/T/ipykernel_76545/3577633952.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


  pose_label                                    image_path  left_elbow_angle  \
0    Goddess  ../../data_clean/Goddess/Images/00000164.jpg        106.676085   
1    Goddess  ../../data_clean/Goddess/Images/00000010.jpg         92.520828   
2    Goddess  ../../data_clean/Goddess/Images/00000004.jpg        252.360775   
3    Goddess  ../../data_clean/Goddess/Images/00000212.jpg        128.659808   
4    Goddess  ../../data_clean/Goddess/Images/00000207.jpg        298.162527   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         256.675469           107.687826            106.745751   
1         276.700754            90.209362             91.378509   
2         108.434949           270.125373            270.125373   
3         243.434949            96.483074             87.510447   
4          63.522157            28.802677             27.520585   

   left_knee_angle  right_knee_angle  hand_angle  left_hip_angle  \
0       236.912390        116.734744  193.258293

/Users/qz/Desktop/Computer Vision/Deep-Learning-Fitness-Poses-Correction-/yoga_env/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/var/folders/_p/cwjtcdzj0w37y_rj10wlyyq00000gn/T/ipykernel_76545/3577633952.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


  pose_label                                  image_path  left_elbow_angle  \
0      Plank  ../../data_clean/Plank/Images/00000428.jpg        171.119341   
1      Plank  ../../data_clean/Plank/Images/00000366.jpg        176.574256   
2      Plank  ../../data_clean/Plank/Images/00000399.jpg        187.943472   
3      Plank  ../../data_clean/Plank/Images/00000010.jpg        190.919950   
4      Plank  ../../data_clean/Plank/Images/00000004.jpg        170.842681   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         173.120056            69.943905            288.529993   
1         181.012188           287.791377             67.075077   
2         182.871363           288.924644             70.951680   
3         187.352379           279.182963             82.460555   
4         166.792408            62.487997            296.876862   

   left_knee_angle  right_knee_angle  hand_angle  left_hip_angle  \
0       187.983940        181.897510  354.319109      175.61

/Users/qz/Desktop/Computer Vision/Deep-Learning-Fitness-Poses-Correction-/yoga_env/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/var/folders/_p/cwjtcdzj0w37y_rj10wlyyq00000gn/T/ipykernel_76545/3577633952.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


  pose_label                                 image_path  left_elbow_angle  \
0       Tree  ../../data_clean/Tree/Images/00000158.jpg        130.399904   
1       Tree  ../../data_clean/Tree/Images/00000170.jpg        170.197503   
2       Tree  ../../data_clean/Tree/Images/00000038.jpg        152.334057   
3       Tree  ../../data_clean/Tree/Images/00000010.jpg        111.789478   
4       Tree  ../../data_clean/Tree/Images/00000004.jpg        311.484397   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         242.429863           171.883357             19.440035   
1         196.927513           197.532728            180.000000   
2         208.950995           186.712329            177.614056   
3         242.981430           167.200223            162.070049   
4          49.472960            35.818011             24.227745   

   left_knee_angle  right_knee_angle  hand_angle  left_hip_angle  \
0       190.681197         25.948389  154.440035      199.400401  

/Users/qz/Desktop/Computer Vision/Deep-Learning-Fitness-Poses-Correction-/yoga_env/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/var/folders/_p/cwjtcdzj0w37y_rj10wlyyq00000gn/T/ipykernel_76545/3577633952.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


  pose_label                                     image_path  left_elbow_angle  \
0   Warrior2  ../../data_clean/Warrior2/Images/00000428.jpg        173.593472   
1   Warrior2  ../../data_clean/Warrior2/Images/00000366.jpg        188.746162   
2   Warrior2  ../../data_clean/Warrior2/Images/00000372.jpg        182.850548   
3   Warrior2  ../../data_clean/Warrior2/Images/00000414.jpg        183.853048   
4   Warrior2  ../../data_clean/Warrior2/Images/00000158.jpg        169.645936   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         171.869898           263.088773            257.550003   
1         172.405357           264.143986            249.775141   
2         175.598200            98.162740            106.958180   
3         175.971736            99.487881            111.997403   
4         174.620298            81.092175             78.087252   

   left_knee_angle  right_knee_angle  hand_angle  left_hip_angle  \
0       181.850988        246.391752  180.

In [15]:
# merge all csv files into one
RESULTS_DIR = f"{angles_path}/"
all_dfs = []

for pose_name in POSES:
    csv_path = os.path.join(RESULTS_DIR, f"{pose_name}.csv")
    df = pd.read_csv(csv_path)

    if "Label" in df.columns and "image_path" not in df.columns:
        df = df.rename(columns={"Label": "image_path"})

    all_dfs.append(df)

# concatenate
df_all = pd.concat(all_dfs, ignore_index=True)
print(df_all.head())
print("Shape:", df_all.shape)

out_csv = os.path.join(RESULTS_DIR, "all_angles.csv")
df_all.to_csv(out_csv, index=False)
print("Saved merged angles to:", out_csv)

  pose_label                                    image_path  left_elbow_angle  \
0    Downdog  ../../data_clean/Downdog/Images/00000372.jpg        195.382277   
1    Downdog  ../../data_clean/Downdog/Images/00000414.jpg        161.394529   
2    Downdog  ../../data_clean/Downdog/Images/00000158.jpg        188.316559   
3    Downdog  ../../data_clean/Downdog/Images/00000164.jpg        163.143669   
4    Downdog  ../../data_clean/Downdog/Images/00000170.jpg        202.094230   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         189.994100           181.919382            180.159746   
1         158.980446           163.869613            198.508847   
2         189.387117           188.434129            168.625045   
3         151.368673           174.269075            190.190288   
4         196.515703           181.847610            181.956953   

   left_knee_angle  right_knee_angle  hand_angle  left_hip_angle  \
0       153.536511        156.576545    1.285724